# Logistic Regression in BQML
Train a simple logistic regression model in BigQuery that predicts a weather description _(Ex: Partly Cloudy)_. The model is trained on data from `weather_replication_public.open_weather_api`. This dataset was generated using weather data from https://openweathermap.org/api. Example code to import this data into BigQuery found [here](https://github.com/FioChuck/weather-ingest). Next the ML model is deployed and used to predict weather descriptions in batch on the `weather_stream.open_weather_api` dataset. This data set contains telemetry weather data with features like humidity, temperature, and date.

In [ ]:
# Sample the training data. Notice this data is labeled with the [desc_long] column
%%bigquery
select * from weather_replication_public.open_weather_api limit 3

Query is running:   0%|          |

Downloading:   0%|          |

,id,lon,lat,desc_short,desc_long,temp,feels_like,temp_min,temp_max,pressure,...,wind_speed,wind_deg,clouds,dt,sunrise,sunset,timezone,city,processing_time,datastream_metadata
0,91470,-84.388,33.749,Clouds,few clouds,289.44,289.42,287.40,290.45,1020.0,...,1.54,10.0,20.0,1695378078,1695381928,1695425718,-14400,Atlanta,2023-09-22 10:25:01.012021+00:00,{'uuid': 'e3acdc05-3996-4eae-bee8-419900000000...
1,38113,-84.388,33.749,Clouds,scattered clouds,289.19,288.91,287.10,290.97,1014.0,...,0.45,308.0,29.0,1678179678,1678190339,1678232313,-18000,Atlanta,2023-03-07 09:05:00.262567+00:00,{'uuid': '4e1c3e13-4245-472b-8a40-64cf00000000...
2,90265,-84.388,33.749,Clear,clear sky,289.82,289.94,287.71,291.42,1015.0,...,0.45,317.0,0.0,1695016613,1695036167,1695080456,-14400,Atlanta,2023-09-18 06:00:00.764078+00:00,{'uuid': 'c86797c4-5fb4-4433-ab6a-02c900000000...


In [ ]:
# Create a logistic regression model and register in Vertex Model Registry
# https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-create-glm
%%bigquery
CREATE OR REPLACE MODEL
  `weather_replication_public.lr_model` OPTIONS ( model_type='LOGISTIC_REG',
    auto_class_weights = TRUE,
    model_registry="vertex_ai",
    # this option registers the model to the Model Registry in Vertex
    input_label_cols = ['desc_long']) AS # select labeled prediction field
SELECT
  TEMP,
  desc_long,
  feels_like,
  temp_min,
  temp_max,
  pressure,
  humidity,
  visibility,
  wind_speed,
  wind_deg,
  clouds,
  CAST(dt AS INT64) AS dt,
  CAST(sunrise AS INT64) AS sunrise,
  CAST(sunset AS INT64) AS sunset,
  timezone,
  EXTRACT(HOUR
  FROM
    processing_time) AS hour,
  EXTRACT(MONTH
  FROM
    processing_time) AS month
FROM
  `weather_replication_public.open_weather_api`

Executing query with job ID: efaf4739-e79c-4dc3-9258-19f4e24886ff
Query executing: 29.83s


ERROR:
 400 Model Registry failed: The Model is deployed or being deployed at the following Endpoint(s): projects/1089470781238/locations/us-central1/endpoints/5966743939707830272. Please undeploy the model before retry.
com.google.cloud.ai.platform.common.errors.AiPlatformException: code=FAILED_PRECONDITION, message=The Model is deployed or being deployed at the following Endpoint(s): projects/1089470781238/locations/us-central1/endpoints/5966743939707830272. Please undeploy the model before retry., cause=null

Location: US
Job ID: efaf4739-e79c-4dc3-9258-19f4e24886ff



In [ ]:
# Sample the unique descriptions from the training data (labeled data). These labels will be predicted by the model using features like humidity and temperature
%%bigquery
SELECT DISTINCT desc_long
FROM `weather_replication_public.open_weather_api`

Query is running:   0%|          |

Downloading:   0%|          |

,desc_long
0,clear sky
1,overcast clouds
2,snow
3,extreme rain
4,light intensity drizzle
5,very heavy rain
6,shower rain
7,broken clouds
8,heavy intensity rain
9,thunderstorm


In [ ]:
# Sample most recent weather sample "current weather" (1 row). This data is used in subsequent steps to predict a label (weather description) for the "current weather"
%%bigquery df_sample
  SELECT temp,
    feels_like,
    temp_min,
    temp_max,
    pressure,
    humidity,
    visibility,
    wind_speed,
    wind_deg,
    clouds,
    dt,
    sunrise,
    sunset,
    timezone,
    EXTRACT(HOUR FROM TIMESTAMP(processing_time)) as hour,
    EXTRACT(MONTH FROM TIMESTAMP(processing_time)) as month,
    TIMESTAMP(processing_time) as processing_time
  FROM `cf-data-analytics.weather_stream.open_weather_api`
  ORDER BY processing_time DESC
  LIMIT 1

Query is running:   0%|          |

Downloading:   0%|          |

In [104]:
# Print most recent weather sample in json format. I used this sample to ping the deployed Vertex endpoint using Postman. When using Postman, make sure to use OAuth2.
import json
json_str = json.dumps(eval(df_sample.to_json(orient='records', lines=True)), indent=4)
print(json_str)

{
    "temp": 293.1300048828,
    "feels_like": 293.0599975586,
    "temp_min": 291.75,
    "temp_max": 294.4100036621,
    "pressure": 1021.0,
    "humidity": 72.0,
    "visibility": 10000.0,
    "wind_speed": 5.6599998474,
    "wind_deg": 60.0,
    "clouds": 0.0,
    "dt": 1695390701,
    "sunrise": 1695381928,
    "sunset": 1695425718,
    "timezone": -14400,
    "hour": 13,
    "month": 9,
    "processing_time": 1695390900387
}


ML.Predict is an exremely powerful too. It allows bulk predictions to be called using existing ETL tooling such as VIEWs and CTAS statements. For example, imagine orchestrating batch predictions using Airflow and the [BigQueryInsertJobOperator](https://airflow.apache.org/docs/apache-airflow-providers-google/stable/operators/cloud/bigquery.html#execute-bigquery-jobs). Examples on my Git [here](https://github.com/FioChuck/cloud-composer-demo).

In [ ]:
# Predict the most recent weather label using ML.Predict in BigQuery.
# https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-predict
%%bigquery
SELECT predicted_desc_long, predicted_desc_long_probs
FROM ML.PREDICT(
        MODEL `weather_replication_public.lr_model`,
        (
          SELECT temp,
              feels_like,
              temp_min,
              temp_max,
              pressure,
              humidity,
              visibility,
              wind_speed,
              wind_deg,
              clouds,
              dt,
              sunrise,
              sunset,
              timezone,
              EXTRACT(HOUR FROM TIMESTAMP(processing_time)) as hour,
              EXTRACT(MONTH FROM TIMESTAMP(processing_time)) as month,
              TIMESTAMP(processing_time) as processing_time
          FROM `cf-data-analytics.weather_stream.open_weather_api`
          ORDER BY processing_time DESC
          LIMIT 1
        )
    )

Query is running:   0%|          |

Downloading:   0%|          |

,predicted_desc_long,predicted_desc_long_probs
0,clear sky,"[{'label': 'clear sky', 'prob': 0.203692194521..."


# Auto ML in BQML

In [ ]:
# Create an AutoML classification model in BigQuery
# https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-create-automl
%%bigquery
CREATE OR REPLACE MODEL `cf-data-analytics.weather_replication_public.automl-model`
OPTIONS(model_type='AUTOML_CLASSIFIER', input_label_cols=['desc_long'], budget_hours=1.0) AS
SELECT
  TEMP,
  desc_long,
  feels_like,
  temp_min,
  temp_max,
  pressure,
  humidity,
  visibility,
  wind_speed,
  wind_deg,
  clouds,
  dt,
  sunrise,
  sunset,
  timezone,
  EXTRACT(HOUR FROM processing_time) as hour,
  EXTRACT(MONTH FROM processing_time) as month
FROM
  `weather_replication_public.open_weather_api`

Query is running:   0%|          |

""


# Auto ML Training Job in Python

In [ ]:
# Create an AutoML classification model in Python using the aiplatform client library
from google.cloud import aiplatform
import requests
import json

In [ ]:
dataset = aiplatform.TabularDataset.create(
    display_name = 'weather_training_data',
    bq_source = 'bq://cf-data-analytics.weather_replication_public.open_weather_api_v')

Creating TabularDataset


INFO:google.cloud.aiplatform.datasets.dataset:Creating TabularDataset


Create TabularDataset backing LRO: projects/1089470781238/locations/us-central1/datasets/8674866367687557120/operations/3048116368458121216


INFO:google.cloud.aiplatform.datasets.dataset:Create TabularDataset backing LRO: projects/1089470781238/locations/us-central1/datasets/8674866367687557120/operations/3048116368458121216


TabularDataset created. Resource name: projects/1089470781238/locations/us-central1/datasets/8674866367687557120


INFO:google.cloud.aiplatform.datasets.dataset:TabularDataset created. Resource name: projects/1089470781238/locations/us-central1/datasets/8674866367687557120


To use this TabularDataset in another session:


INFO:google.cloud.aiplatform.datasets.dataset:To use this TabularDataset in another session:


ds = aiplatform.TabularDataset('projects/1089470781238/locations/us-central1/datasets/8674866367687557120')


INFO:google.cloud.aiplatform.datasets.dataset:ds = aiplatform.TabularDataset('projects/1089470781238/locations/us-central1/datasets/8674866367687557120')


# Call deployed Auto ML Model
Model deployed using Vertex AI UI

In [86]:
from typing import Dict, List, Union

from google.cloud import aiplatform
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value


def predict_custom_trained_model_sample(
    project: str,
    endpoint_id: str,
    instances: Union[Dict, List[Dict]],
    location: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com",
):
    """
    `instances` can be either single instance of type dict or a list
    of instances.
    """
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)
    # The format of each instance should conform to the deployed model's prediction input schema.
    instances = instances if isinstance(instances, list) else [instances]
    instances = [
        json_format.ParseDict(instance_dict, Value()) for instance_dict in instances
    ]
    parameters_dict = {}
    parameters = json_format.ParseDict(parameters_dict, Value())
    endpoint = client.endpoint_path(
        project=project, location=location, endpoint=endpoint_id
    )
    response = client.predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    # The predictions are a google.protobuf.Value representation of the model's predictions.
    predictions = response.predictions
    for prediction in predictions:
        print(" prediction:", dict(prediction))
    return predictions


In [ ]:
# https://github.com/googleapis/python-aiplatform#automls

job = aiplatform.AutoMLTabularTrainingJob(
  display_name="train-automl",
  optimization_prediction_type="classification")

model = job.run(
    dataset=dataset,
    target_column="desc_long",
    training_fraction_split=0.6,
    validation_fraction_split=0.2,
    test_fraction_split=0.2,
    budget_milli_node_hours=1000,
    model_display_name="my-automl-model",
    disable_early_stopping=False,
)

No column transformations provided, so now retrieving columns from dataset in order to set default column transformations.


INFO:google.cloud.aiplatform.training_jobs:No column transformations provided, so now retrieving columns from dataset in order to set default column transformations.


The column transformation of type 'auto' was set for the following columns: ['visivility', 'lon', 'wind_speed', 'wind_deg', 'humidity', 'temp_max', 'month', 'processing_time', 'hour', 'feels_like', 'dt', 'sunset', 'timezone', 'temp_min', 'city', 'TEMP', 'clouds', 'lat', 'pressure', 'sunrise'].


INFO:google.cloud.aiplatform.training_jobs:The column transformation of type 'auto' was set for the following columns: ['visivility', 'lon', 'wind_speed', 'wind_deg', 'humidity', 'temp_max', 'month', 'processing_time', 'hour', 'feels_like', 'dt', 'sunset', 'timezone', 'temp_min', 'city', 'TEMP', 'clouds', 'lat', 'pressure', 'sunrise'].


View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/4044894714976010240?project=1089470781238


...

INFO:google.cloud.aiplatform.training_jobs:AutoMLTabularTrainingJob projects/1089470781238/locations/us-central1/trainingPipelines/4044894714976010240 current state:
PipelineState.PIPELINE_STATE_RUNNING


AutoMLTabularTrainingJob run completed. Resource name: projects/1089470781238/locations/us-central1/trainingPipelines/4044894714976010240


INFO:google.cloud.aiplatform.training_jobs:AutoMLTabularTrainingJob run completed. Resource name: projects/1089470781238/locations/us-central1/trainingPipelines/4044894714976010240


Model available at projects/1089470781238/locations/us-central1/models/7004124364421464064


INFO:google.cloud.aiplatform.training_jobs:Model available at projects/1089470781238/locations/us-central1/models/7004124364421464064


In [ ]:
%%bigquery df
SELECT
  CAST(lon AS FLOAT64) AS lon,
  CAST(lat AS FLOAT64) AS lat,
  CAST(TEMP AS FLOAT64) AS TEMP,
  CAST(feels_like AS FLOAT64) AS feels_like,
  CAST(temp_min AS FLOAT64) AS temp_min,
  CAST(temp_max AS FLOAT64) AS temp_max,
  CAST(pressure AS FLOAT64) AS pressure,
  CAST(humidity AS FLOAT64) AS humidity,
  CAST(visibility AS FLOAT64) AS visivility,
  CAST(wind_deg AS FLOAT64) AS wind_deg,
  CAST(clouds AS FLOAT64) AS clouds,
  CAST(dt AS STRING) AS dt,
  CAST(sunrise AS STRING) AS sunrise,
  CAST(sunset AS STRING) AS sunset,
  CAST(timezone AS FLOAT64) AS timezone,
  CAST(city AS STRING) AS city,
  CONCAT(processing_time,"+0000") AS processing_time,
  CAST(wind_speed AS FLOAT64) AS wind_speed,
  CAST(EXTRACT(HOUR
    FROM
      TIMESTAMP(processing_time)) AS STRING) AS hour,
  CAST(EXTRACT(MONTH
    FROM
      TIMESTAMP(processing_time)) AS STRING) AS month,
FROM
  `cf-data-analytics.weather_stream.open_weather_api`
ORDER BY
  processing_time DESC
LIMIT
  1

Query is running:   0%|          |

Downloading:   0%|          |

In [115]:
# Payload created from SQL Query and passed into the Deployed Endpoint
payload = df.to_dict(orient="records")
payload

[{'lon': -84.38800048828125,
  'lat': 33.749000549316406,
  'TEMP': 293.6499938964844,
  'feels_like': 293.6600036621094,
  'temp_min': 292.4100036621094,
  'temp_max': 295.1199951171875,
  'pressure': 1021.0,
  'humidity': 73.0,
  'visivility': 10000.0,
  'wind_deg': 60.0,
  'clouds': 0.0,
  'dt': '1695391327',
  'sunrise': '1695381928',
  'sunset': '1695425718',
  'timezone': -14400.0,
  'city': 'Atlanta',
  'processing_time': '2023-09-22 14:10:00.785892+0000',
  'wind_speed': 4.119999885559082,
  'hour': '14',
  'month': '9'}]

In [99]:
output = predict_custom_trained_model_sample(
    project="1089470781238",
    endpoint_id="3703403651977445376",
    location="us-central1",
    instances=payload)

response
 deployed_model_id: 6341536667291090944
 prediction: {'classes': ['clear sky', 'overcast clouds', 'broken clouds', 'mist', 'few clouds', 'scattered clouds', 'moderate rain', 'light rain', 'thunderstorm', 'haze', 'heavy intensity rain', 'fog', 'smoke', 'light intensity drizzle', 'thunderstorm with light rain', 'thunderstorm with heavy rain', 'thunderstorm with rain', 'snow', 'shower rain', 'very heavy rain', 'light snow', 'squalls', 'extreme rain'], 'scores': [0.9966756105422974, 0.0001399543107254431, 2.580775415594871e-08, 3.406922769499943e-05, 8.682216866873205e-05, 2.015008782052519e-08, 6.780232070013881e-06, 0.0001107563293771818, 2.622883403091691e-05, 7.791261305101216e-05, 2.045238488790346e-06, 1.748060896034076e-08, 0.00283790985122323, 3.214673256479728e-07, 2.844444679794833e-07, 3.203337328727685e-08, 5.290877993502363e-07, 5.89933328853931e-08, 1.928181490029601e-07, 3.308540215130051e-07, 2.731092330066076e-08, 7.184029726658991e-08, 2.285624667308639e-08]}


In [116]:
# Latest weather description prediction shown below. Does this match the current weather in Atlanta?
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(dict(output[0]))


{   'classes': ['clear sky', 'overcast clouds', 'broken clouds', 'mist', 'few clouds', 'scattered clouds', 'moderate rain', 'light rain', 'thunderstorm', 'haze', 'heavy intensity rain', 'fog', 'smoke', 'light intensity drizzle', 'thunderstorm with light rain', 'thunderstorm with heavy rain', 'thunderstorm with rain', 'snow', 'shower rain', 'very heavy rain', 'light snow', 'squalls', 'extreme rain'],
    'scores': [0.9966756105422974, 0.0001399543107254431, 2.580775415594871e-08, 3.406922769499943e-05, 8.682216866873205e-05, 2.015008782052519e-08, 6.780232070013881e-06, 0.0001107563293771818, 2.622883403091691e-05, 7.791261305101216e-05, 2.045238488790346e-06, 1.748060896034076e-08, 0.00283790985122323, 3.214673256479728e-07, 2.844444679794833e-07, 3.203337328727685e-08, 5.290877993502363e-07, 5.89933328853931e-08, 1.928181490029601e-07, 3.308540215130051e-07, 2.731092330066076e-08, 7.184029726658991e-08, 2.285624667308639e-08]}
